In [2]:
#!pip install opencv-python
#!pip install playsound
#!pip install viola
#!pip install voila
!jupyter serverextension enable -sys-prefix voila

Enable a serverextension in configuration.

Usage
    jupyter serverextension enable [--system|--sys-prefix]

Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--user

    Perform the operation for the current user

--system

    Perform the operation system-wide

--sys-prefix

    Use sys.prefix as the prefix for installing server extensions

--py

    Install from a Python package

--python

    Install from a Python package
--log-level=<Enum> (Application.log_level)

    Default: 30

    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')

    Set the log level by value or name.

--config=<Unicode> (JupyterApp.config_file)

    Default: ''

    Full path of a config file.

To see all available configurables, use `--help-all`



Bad config encountered during initialization:
Unrecognized flag: '-sys-prefix'


In [ ]:
import playsound
import numpy as np
import argparse
from playsound import playsound
import cv2 as cv
import seaborn as sns
import matplotlib.pyplot as plt
import time
import cv2 as cv
import pygame
from pygame import mixer
pygame.init()

In [ ]:
playsound('d.mp3',"**0**")
playsound('d.mp3',"**0**")

In [ ]:
playsound('hi.mp3',"**0**")
playsound('hands_breath.mp3',False)

In [ ]:


net=cv.dnn.readNetFromTensorflow('graph_opt.pb')


BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

inWidth = 368
inHeight = 368
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

#cap = cv.VideoCapture('yoga.mp4')

cap = cv.VideoCapture(0)

In [ ]:
body_parts=[3]
poinst_history =np.array([(16,30),(20,20)]
r=15
body_parts=[3,6]    
k=0

for i in range(202,530):
    poinst_history.append([(0,0),(0,0)])
    frame = np.load('temp\\frame' + str(i) + '.npy')
    cv.waitKey(1)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (10.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :39, :, :]  
    points=[]


    for j in range(len(body_parts)):
        heatMap = out[0, body_parts[j], :, :]
        pre_point =  poinst_history[-2][j]   
       
        x,y =pre_point[0],pre_point[1]
        local_area=heatMap[ y-r:y+r,x-r:x+r]
        _, conf, _, local_point = cv.minMaxLoc(local_area)
        
        point=(local_point[0] + x -r, local_point[1] + y -r)
        print(x,y)
        print(local_point)
        print(point)
        poinst_history[-1][j]=point
        frame_x = (frameWidth * point[0]) / out.shape[3]
        frame_y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(frame_x), int(frame_y)) if conf > 0 else None)
    for p in points:
        cv.ellipse(frame, p, (10, 10), 0, 0, 180, (0, 0, 255), cv.FILLED)
    
    t, _ = net.getPerfProfile()
    cv.putText(frame, str(i), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    cv.imshow('OpenPose using OpenCV', frame) 
    r=2
    cv.waitKey(100)
cv.destroyAllWindows()


In [ ]:
import cv2 as cv
mixer.music.load('rain2.mp3')
vol=0.01
mixer.music.set_volume(0.01)
mixer.music.play(200)
mi_max=cv.minMaxLoc(np.array(poinst_history)[:,1,1])
ups_frames=mi_max[3][1]-mi_max[2][1]
b_in=True


k=0
st,end=202,530
for i in range(st,end):
    vol=vol+0.01
    mixer.music.set_volume(vol)
    points=[]
    frame = np.load('temp\\frame' + str(i) + '.npy')
    cv.waitKey(1)
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
  
    if (i>st+ ups_frames):
        st = i 
        vol=0.01
        if (b_in):
            mixer.music.load('rain.mp3')
        else: 
            mixer.music.load('rain2.mp3')
        b_in = not(b_in)
        mixer.music.play(20)
        
    

    for j in range(len(body_parts)):
       
        point=poinst_history[k][j]
        frame_x = (frameWidth * point[0]) / out.shape[3]
        frame_y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(frame_x), int(frame_y)) if conf > 0 else None)
    for p in points:
        cv.ellipse(frame, p, (10, 10), 0, 0, 180, (0, 0, 255), cv.FILLED)
    
    t, _ = net.getPerfProfile()
    cv.putText(frame, str(i-st), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    cv.imshow('OpenPose using OpenCV', frame) 
    r=2
    k=k+1
    cv.waitKey(200)
cv.destroyAllWindows()
mixer.music.stop()



In [ ]:
mixer.music.load('rain2.mp3')
for i in range(10):    
    mixer.music.play(1)
    mixer.music.set_volume(0.01)


In [ ]:
not(b_in)

In [ ]:
cv.destroyAllWindows()